In [ ]:
from pyspark_llap import HiveWarehouseSession
hive = HiveWarehouseSession.session(spark).build()
hive.showDatabases().show(10)

In [ ]:
hive.setDatabase("default")
hive.createTable("web_sales").ifNotExists().\
    column("sold_time_sk", "bigint").column("ws_ship_date_sk", "bigint").create()
hive.describeTable("web_sales").show()
hive.dropTable("web_sales", True, True)

In [ ]:
df = spark.read.csv("/data/examples/Employees.tsv", 
                    header=False, sep='\t', mode="DROPMALFORMED", inferSchema=True
                   ).toDF("name", "title", "department", "salary")
df.printSchema()

In [ ]:
df.write.format(HiveWarehouseSession().HIVE_WAREHOUSE_CONNECTOR)\
    .mode("overwrite").option("table", "employees").save()

In [ ]:
df = hive.executeQuery("select * from employees")
df.toPandas().head(10)